In [ ]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
earlystopping_callback = EarlyStopping(monitor='loss',verbose=1,patience=12,baseline=None)
from pandas_datareader import data
from datetime import datetime
stocks = ['ADSK','AMZN','F','BTC-GBP','BTC','WIX','INTU', 'SQ','AAPL', 'MSFT', 'GOOG','FB', 'TSLA', 'TSM', 'NVDA', 
          'BABA', 'ASML', 'PYPL', 'NFLX','CSCO','ADSK']
stocks2 = ['WU', 'WATT']

for st in stocks: 
    df = data.DataReader(st,  'yahoo', datetime(2012, 1, 1), datetime(2021, 10, 20))
    df.info()
    df['Date'] = df.index
    df['Date'] = pd.to_datetime(df['Date'])
    df = df[~df.Date.between(datetime(2020, 2, 1),datetime(2020, 7, 1))]
    df = df[~df.Date.between(datetime(2021, 10, 19),datetime(2021, 10, 21))]
    df.reset_index(drop=True, inplace=True)
    df.corr()
    df.set_axis(df['Date'], inplace=True)
    df.drop(columns=['Open', 'High', 'Low', 'Volume'], inplace=True)
    import plotly
    import plotly.graph_objects as go
    print("Plotly Version: ",plotly.__version__)

    trace = go.Scatter(
        x = df['Date'],
        y = df['Close'],
        mode = 'lines',
        name = 'Data'
    )
    layout = go.Layout(
        title = "",
        xaxis = {'title' : "Date"},
        yaxis = {'title' : "Close (Dollars)"}
    )
    fig = go.Figure(data=[trace], layout=layout)
   # fig.show()

    close_data = df['Close'].values
    close_data = close_data.reshape((-1,1))

    split_percent = 0.95
    split = int(split_percent*len(close_data))

    close_train = close_data[:split]
    close_test = close_data[split:]

    date_train = df['Date'][:split]
    date_test = df['Date'][split:]

    print(len(close_train))
    print(len(close_test))

    look_back = 7

    train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
    test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

    from keras.models import Sequential
    from keras.layers import LSTM, Dense

    model = Sequential()
    model.add(
        LSTM(32,
            activation='relu',
            input_shape=(look_back,1))
    )
    model.add(Dropout(0.05))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    num_epochs = 50
    model.fit_generator(train_generator, epochs=num_epochs, verbose=0, callbacks=[earlystopping_callback])

    prediction = model.predict_generator(test_generator)
    # prediction

    close_train = close_train.reshape((-1))
    close_test = close_test.reshape((-1))
    prediction = prediction.reshape((-1))


    trace1 = go.Scatter(
        x = date_train,
        y = close_train,
        mode = 'lines',
        name = 'Data'
    )
    trace2 = go.Scatter(
        x = date_test,
        y = prediction,
        mode = 'lines',
        name = 'Prediction'
    )
    trace3 = go.Scatter(
        x = date_test,
        y = close_test,
        mode='lines',
        name = 'Ground Truth'
    )
    layout = go.Layout(
        title = str(st),
        xaxis = {'title' : "Date"},
        yaxis = {'title' : "Close"}
    )
    fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
    fig.update_layout(xaxis_range=[datetime(2021, 5, 1),
                               datetime(2021, 12, 1)])
    
    fig.update_layout(yaxis_range=[0,max(close_data)*1.1])
    
    fig.show()
    fig.write_image("IMAGES/" + str(st) + "1.png")
    
    close_data = close_data.reshape((-1))

    def predict(num_prediction, model):
        prediction_list = close_data[-look_back:]

        for _ in range(num_prediction):
            x = prediction_list[-look_back:]
            x = x.reshape((1, look_back, 1))
            out = model.predict(x)[0][0]
            prediction_list = np.append(prediction_list, out)
        prediction_list = prediction_list[look_back-1:]

        return prediction_list

    def predict_dates(num_prediction):
        last_date = df['Date'].values[-1]
        prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
        return prediction_dates

    num_prediction = 14
    forecast = predict(num_prediction, model)
    forecast_dates = predict_dates(num_prediction)

    trace1 = go.Scatter(
        x = df['Date'].tolist(),
        y = close_data,
        mode = 'lines',
        name = 'Data'
    )
    trace2 = go.Scatter(
        x = forecast_dates,
        y = forecast,
        mode = 'lines',
        name = 'Prediction'
    )
    layout = go.Layout(
        title = str(st),
        xaxis = {'title' : "Date"},
        yaxis = {'title' : "Close"}
    )

    fig = go.Figure(data=[trace1, trace2], layout=layout)
    fig.update_layout(xaxis_range=[datetime(2021, 10, 10),
                               datetime(2021, 10, 30)])
    fig.update_layout(yaxis_range=[0.3*max(close_data),max(close_data)])
    
    fig.show()
    fig.write_image("IMAGES//" + str(st) + "2.png")

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2467 entries, 2012-01-03 to 2021-10-20
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   High       2467 non-null   float64
 1   Low        2467 non-null   float64
 2   Open       2467 non-null   float64
 3   Close      2467 non-null   float64
 4   Volume     2467 non-null   int64  
 5   Adj Close  2467 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 134.9 KB
Plotly Version:  5.3.1
2242
118


C:\Users\SparkStorms\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py:1972: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



Epoch 00031: early stopping


C:\Users\SparkStorms\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py:2035: UserWarning:

`Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.



<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2467 entries, 2012-01-03 to 2021-10-20
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   High       2467 non-null   float64
 1   Low        2467 non-null   float64
 2   Open       2467 non-null   float64
 3   Close      2467 non-null   float64
 4   Volume     2467 non-null   int64  
 5   Adj Close  2467 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 134.9 KB
Plotly Version:  5.3.1
2242
118


C:\Users\SparkStorms\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py:1972: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



Epoch 00025: early stopping


C:\Users\SparkStorms\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py:2035: UserWarning:

`Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.

